In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import sys
from sklearn.model_selection import KFold
from tqdm.notebook import tqdm
from catboost import CatBoostRegressor
from bs4 import BeautifulSoup    
import requests 

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.feature_selection import f_regression, mutual_info_regression
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler, PolynomialFeatures

from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from pandas import Series

from sklearn.metrics import recall_score
from sklearn.metrics import auc, roc_auc_score, roc_curve
from sklearn.base import clone
import datetime
from datetime import datetime, timedelta
import math

from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import BaggingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from sklearn.ensemble import GradientBoostingRegressor, AdaBoostRegressor
from pprint import pprint

In [ ]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

In [ ]:
RANDOM_SEED = 42

In [ ]:
def mape(y_true, y_pred):
    return np.mean(np.abs((y_pred-y_true)/y_true))

# Часть 1. Парсинг

Для начала посмотрим тестовый датасет

In [ ]:
DIR_TEST   = '../input/sf-dst-car-price-prediction/'
test = pd.read_csv(DIR_TEST+'test.csv')
sample_submission = pd.read_csv(DIR_TEST+'sample_submission.csv')

In [ ]:
test.sample(3)

In [ ]:
test.info()

Видим 32 столбца, из них только 6 - числовые.  
Т.е. у нас преимущественно категориальный датасет.  
Посмотрим на наиболее типичные значения в разных колонках тестового датасета, чтобы понимать, как строить тренировочный датасет train из того, что напарсим.

In [ ]:
#for col in test.columns:
#    if col != 'complectation_dict':
#        print(f'name of column: {col}','\n', test[col].value_counts()[:10], '\n')

Для подготовки к парсингу создадим словарь brand_dict, где ключи - марки автомобилей, а значения - их кол-ва в test:

In [ ]:
brands = test.brand.value_counts()
brand_dict = {brands.index[i]: brands[i] for i in range(len(brands))}
brand_dict

# Сделаем парсинг сайта авто.ру следующим образом:  
 берем фильтр по Москве легковые с пробегом  
 берем фильтр по марке автомобиля из словаря brand_dict, другие марки не трогаем  
 т.к. на каждой странице сайта авто.ру 38 объявления, то парсим примерно brand_dict.keys10/38 страниц,   т.е. в 10 раз больше объявлений, чем в test для каждой марки(если найдется столько)  
 не заходим на страницы отдельных объявлений, т.к. не будем использовать инфо о спецоборудованиии и тд,   ограничимся основными данными 

Ниже приведен основная часть кода для парсинга

In [ ]:
'''auto_list_long =[] # здесь соберем список списков(эл-т - список параметров конкретного автомобиля)
for brand in ['MERCEDES']:  # марка авто
    
    for i in range(1, 1200):     # кол-во страниц для парсинга с авто.ру   
        response = requests.get(f'https://auto.ru/moskva/cars/{brand}/used/?output_type=list&page={i}')
        if response.status_code != 200:
            raise BaseException("response code" + str(response.status_code))
        response.encoding = 'utf-8'
        soup = BeautifulSoup(response.text, 'html.parser')
        page = soup.find_all('div', class_='ListingItem-module__container')  
        for k in range(len(page)):     # цикл для прохода по объявлениям в одной странице
            auto_html = page[k].find_all('meta')
            auto_list = []
            for i in range(len(auto_html)):   # цикл для обработки отдельного объявления на странице
                auto_list.append((str(auto_html[i])[15:]).split('"', 1)[0])
                
            # Убираем лишние параметры автомобиля из списка    
            pos_out = {4,11,12,14,15,19}
            auto_list_short = []
            for n in range(len(auto_list)):   # цикл для удаления лишних параметров из объявления
                if n not in pos_out:
                    auto_list_short.append(auto_list[n].replace('\xa0', ' ')) 
        
            # Считываем пробег(он расположен отдельно от других данных)    
            all_km = page[k].find('div', class_='ListingItem-module__kmAge')
            xx = str(all_km)[39:].split('<', 1)[0].replace('\xa0', ' ')
            auto_list_short.append(str(xx))    # добавляем пробег
            auto_list_long.append(auto_list_short)    # добавляем список пар-ров автомобиля в список списков
'''

Код запускался не из Kaggle.  
После получения списка списков по каждому отдельному бренду он преобразовывался в датафрейм.  
Затем 12 датафреймов были склеены в один - искомый train.  
Загрузим его и сделаем предобработку, чтобы соотнести полученные признаки с признаками из test 

# Часть 2. Предобработка данных.

In [ ]:
DIR_TRAIN   = '../input/car-price-train/'
train = pd.read_csv(DIR_TRAIN+'my_csv_export.csv')

In [ ]:
train.shape

In [ ]:
train.sample(3)

In [ ]:
train.info()

Видим 14 признаков плюс целевая переменная 'price'  
Для начала приведем в соответствие с test численные столбцы

In [ ]:
# Меняем тип переменных на int64 для численных признаков и целевой и убираем пропуски
num_f = ['modelDate', 'productionDate', 'numberOfDoors','mileage', 'price']
train['mileage'] = train['mileage'].apply(lambda x: int(x.replace(' ', '').replace('км', '')))
train.dropna(subset=num_f, inplace=True)

for col in num_f:
    train[col] = train[col].astype('int64')

In [ ]:
# train уменьшился примерно на 6700 строк, в основном - из-за пустых значений в 'price'
train.shape

In [ ]:
# Добавим в train признак 'vendor', который есть в test
eur = ['BMW', 'VOLKSWAGEN', 'MERCEDES', 'AUDI', 'SKODA', 'VOLVO']
train['vendor'] = train['brand'].apply(lambda x: 'EUROPEAN' if x in eur else 'JAPANESE')

Сравнивая test и train, видим, что только признаки 'name' и 'model_name' имеют разные области значений, с остальными все ок

In [ ]:
#  Признак 'name'
display(test.name.sample(3), train.name.sample(3)) 

Видим, что признак 'name' - это линейная комбинация других признаков:  
'name' = 'engineDisplacement' + 'vehicleTransmission' + bool(Allroad) из 'vehicleConfiguration'  
Понятно, что это категориальный признак и линейная зависимость тут не играет роли, но унифицировать его долго, и это явно не самый важный признак, т.к. его компоненты не теряются.  
Поэтому не включаем его в итоговый датасет.  
'Allroad' как особо ценную опцию позже превратим в новый бинарный признак.

In [ ]:
#  Признак 'model_name'
display(test.model_name.value_counts()[:3], train.model_name.value_counts()[:3]) 

Видим, что в train этот признак более информативный, но нас интересует короткое название модели,  
выделим его

In [ ]:
# Сохраняем исходный вид признаков
train['model_name_long'] =  train['model_name'].copy()
test['model_name_long'] =  test['model_name'].copy()

In [ ]:
# Вырезаем названия моделей в train и меняем прописные буквы на заглавные 
train['model_name'] = train['model_name'].apply(lambda x: (x.split(' ')[1]).upper())

In [ ]:
# Укорачиваем двойные названия
train['model_name'] = train['model_name'].apply(lambda x: x.split('-')[0])
test['model_name'] = test['model_name'].apply(lambda x: x.split('_')[0])

In [ ]:
# Укорачиваем названия моделей, начинающися с цифры( это надо в основном для 'BMW') 
num_list = ['0','1','2','3','4','5','6','7','8','9']
train['model_name'] = train['model_name'].apply(lambda x: x[0] if x[0] in num_list else x)
test['model_name'] = test['model_name'].apply(lambda x: x[0] if x[0] in num_list else x)

Проверяем, видим, что совпадение хорошее. Теперь все признаки в train  и test в основном совпадают в областях значений.

In [ ]:
display(np.sort(test[test.brand == 'MERCEDES'].model_name.unique()),
        np.sort(train[train.brand == 'MERCEDES'].model_name.unique()))

Выбираем признаки для дальнейшей работы  

In [ ]:
columns = ['bodyType', 'brand', 'color', 'fuelType', 'modelDate', 'model_name', 'numberOfDoors',
           'productionDate', 'vehicleConfiguration', 'vehicleTransmission', 'engineDisplacement',
           'enginePower', 'mileage', 'vendor']
cars_train = train[columns]
cars_test = test[columns]

 Ставим метки 'sample'= 1 or 0 для различения train и test и склеиваем выбранные столбцы.  
 Назовем полученный датасет cars, он будет использоваться для исследовательского анализа признаков(EDA)

In [ ]:
cars_train['sample'] = 1
cars_test['sample'] = 0
cars = cars_train.append(cars_test, ignore_index=True, sort=False)

# Целевая переменная
y = train['price']

# Часть 3. EDA.

In [ ]:
cars.sample(3)

In [ ]:
cars.info()

Мы видим 14 признаков('sample' не считаем), из них 3 - числовых и 11 - категориальных.  
Сразу заметим, что два категориальных признака - 'enginePower' и 'engineDisplacement' по сути являются числовыми, принимая значения в своих числовых диапазонах. Понятно также то, что цена автомобиля растет в зависимости от роста мощности двигателя или его объема. Поэтому уже на этапе анализа сделаем эти признаки числовыми.

In [ ]:
# Количество уникальных значений в этих признаках достаточно большое
print(cars['enginePower'].nunique(), cars['engineDisplacement'].nunique())

In [ ]:
# Избавляемся от пустых значений в 'engineDisplacement'(их всего 101), заменяя на среднее
cars['engineDisplacement'] = cars['engineDisplacement'].apply(lambda x: '2.5 LTR' if x == ' LTR' else x)

In [ ]:
cars['enginePower'] = cars['enginePower'].apply(lambda x: int(x.split(' ')[0].replace('.', '')))
cars['engineDisplacement'] = cars['engineDisplacement'].apply(
                                                    lambda x: int(x.split(' ')[0].replace('.', '')))

In [ ]:
# Списки числовых и категориальных признаков в cars
num_f = ['modelDate', 'productionDate', 'mileage', 'enginePower', 'engineDisplacement']
cat_f = ['bodyType', 'brand', 'color', 'fuelType', 'model_name', 'numberOfDoors', 'vehicleConfiguration',
         'vehicleTransmission', 'vendor']

# 3.1 Числовые признаки

In [ ]:
for i in num_f:
    plt.figure()
    sns.distplot(cars[i])
    plt.title(i)
    plt.show()

Видим, что распределения у признаков близки к нормальным, справа или слева имеются длинные хвосты.  
Посмотрим на выбросы

In [ ]:
def outliers_iqr(x):
    ''' found outliers '''
    quartile_1, quartile_3 = np.percentile(x, [25, 75])
    iqr = quartile_3 - quartile_1
    lower_bound = quartile_1 - (iqr * 1.5)
    upper_bound = quartile_3 + (iqr * 1.5)    
    return np.where((x > upper_bound) | (x < lower_bound))[0]

In [ ]:
# Выведем на экран количество выбросов по межквартильному размаху
for i in num_f:
    print(i, len(outliers_iqr(cars[i])))

По межквартильному размаху выбросов оказалось очень много. Имеет смысл ввести эмпирические границы, чтобы отсечь далекие хвосты в данных. Однако это ухудшило оценку и в дальнейшем не использовалось.

In [ ]:
# Посмотрим корреляцию числовых признаков
correlation = cars[num_f].corr()
plt.figure(figsize=(10, 6))
sns.heatmap(correlation, annot=True, cmap='coolwarm')

Выводы по рассмотрению корреляции числовых признаков:  
  +0.97 между годом выпуска и годом модели, видимо, имеет смысл один из этих  признаков убрать или заменить на их разность  
  +0.84 между мощностью и объемом двигателя  
  -0.79 между пробегом и годом производства, здесь тоже можно создать новый признак вроде пробег/возраст

# 3.2 Категориальные признаки

In [ ]:
for col in cat_f:
    print(col, cars[col].nunique(), '\n', cars[col].value_counts()[:3], '\n')

Зфметим, что признаки 'model_name' и  'vehicleConfiguration' имеют очень большой спектр значений(438 и 650)   
Остальные 7 категориальных признаков имеют небольшие спектры значений.  
Займемся пока созданием новых признаков, потом продолжим анализ старых и новых вместе. 

# Часть 4. Feature Engineering

Создадим два бинарных категориальных признака, которые на практике сильно влияют на стоимость автомобиля:  
'allroad' - будет отражать наличие полного привода  
'luxury' - будет отражать премиальность бренда

In [ ]:
cars['luxury'] = cars['brand'].apply(lambda x: 1 if x in ['BMW','MERCEDES','LEXUS','INFINITI'] else 0)
cat_f.append('luxury')

In [ ]:
cars['allroad'] = cars['vehicleConfiguration'].apply(lambda x: 1 if x.split('_')[0] == 'ALLROAD' else 0)
cat_f.append('allroad')

Из числовых, как и планировали выше, сделаем:  
    'model_age' - новизна модели на момент производства  
    'km_pro_year' - cтепень эксплуатации(пробег/год)

In [ ]:
cars['model_age'] = cars['productionDate'] - cars['modelDate']
cars['km_pro_year'] = (cars['mileage']/(2021 - cars['productionDate'])).astype('int64')
num_f.append('model_age')
num_f.append('km_pro_year')

Построим их гистограммы, отсекая для наглядности хвосты справа

In [ ]:
plt.figure()
sns.distplot(cars['model_age'][cars['model_age'] < 10])
plt.title('model_age')
plt.show()

In [ ]:
plt.figure()
sns.distplot(cars['km_pro_year'][cars['km_pro_year'] < 35000])
plt.title('km_pro_year')
plt.show()

In [ ]:
# Снова построим heatmap. Видим, что новые признаки не имеют сильных корреляций
correlation = cars[num_f].corr()
plt.figure(figsize=(10, 6))
sns.heatmap(correlation, annot=True, cmap='coolwarm')

In [ ]:
# Значимость числовых признаков для целевой по f_regression (линейная зависимость):
imp_num = Series(f_regression(cars[cars['sample'] == 1][num_f], y)[0], index = num_f)
imp_num.sort_values(inplace = True)
imp_num.plot(kind = 'barh')

Создадим полиномиальные новые признаки на основе двух числовых 'productionDate' и 'enginePower' и нормируем их

In [ ]:
pf = PolynomialFeatures(3)
poly_features = pf.fit_transform(cars[[ 'productionDate', 'enginePower']])
poly = pd.DataFrame(poly_features).drop([0,1,2,3,4,6,8], axis = 1)
poly.columns = ['date_power5', 'date_power7', 'date_power9']

for col in list(poly.columns):
    col_max = poly[col].max()    
    poly[col] = poly[col].apply(lambda x: int(round(x/col_max*100000, 0)))
    
cars = pd.concat([cars, poly], axis=1)
num_f = num_f + list(poly.columns)

In [ ]:
# Проверим значимость полиномиальных признаков
imp_num = Series(f_regression(cars[cars['sample'] == 1][num_f], y)[0], index = num_f)
imp_num.sort_values(inplace = True)
imp_num.plot(kind = 'barh')

Полиномиальные признаки показывают высокую значимость для целевой переменной, но  
они очень сильно скоррелированны со своими предшественниками.
Попробуем в дальнейшем запускать модели с ними и без них

In [ ]:
correlation = cars[num_f].corr()
plt.figure(figsize=(10, 6))
sns.heatmap(correlation, annot=True, cmap='coolwarm')

In [ ]:
cars.info()

Итоги  
 В результате EDA и Feature Engineering добавлены:  
  2 новых бинарных категориальных признака  
  5 новых числовых, 3 из которых - полиномиальные  
  2 новых числовых, переделанных из категориальных  
  Выбросы не удалялись  
  Датасет содержит 10 числовых и 11 категориальных признаков  
  Сохранены сильно скоррелированные признаки  для отбора в ходе тестирования моделей

# 5. Label Encoding

In [ ]:
# Кодируем категориальные признаки
for col in cat_f:
    cars[col] = cars[col].astype('category').cat.codes

In [ ]:
X_train_test = cars.copy()
X_train_test.head(3)

In [ ]:
# Разделяем обратно train и test
train = X_train_test.query('sample == 1').drop(['sample'], axis=1)
test = X_train_test.query('sample == 0').drop(['sample'], axis=1)

In [ ]:
# Делим train для обучения и проверки моделей
X_train, X_test, y_train, y_test = train_test_split(train, y, test_size=0.2, shuffle=True, random_state=42)

# Model 1. "Наивная"  
Эта модель будет предсказывать среднюю цену по модели двигателя (enginePower). 
C ней будем сравнивать другие модели.

In [ ]:
tmp_train = X_train.copy()
tmp_train['price'] = y_train

In [ ]:
# Находим median по экземплярам enginePower в трейне и размечаем тест
predict = X_test['enginePower'].map(tmp_train.groupby('enginePower')['price'].median())

#оцениваем точность
print(f"Точность наивной модели по метрике MAPE: {(mape(y_test, predict.values))*100:0.2f}%")

# Model 2. CatBoost

Из параметров CatBoost удалось подобрать оптимальный  learning_rate=0.25 при кол-ве итераций 10000.  
bagging_temperature и random_strength на результат не повлияли.  
Объявление категориальных признаков(неоцифрованных) в пар-р cat_features тоже не улучшило результат 

In [ ]:
ITERATIONS = 10000
LR         = 0.25
model = CatBoostRegressor(iterations = ITERATIONS,
                          learning_rate = LR,
                          random_seed = RANDOM_SEED,
                          eval_metric='MAPE',
                          custom_metric=['R2', 'MAE']
                         )
model.fit(X_train, y_train,
#         cat_features=cat_f,
         eval_set=(X_test, y_test),
         verbose_eval=100,
         use_best_model=True,
         plot=True
         )

In [ ]:
predict = model.predict(X_test)

# оцениваем точность
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict))*100:0.3f}%")

MAPE = 1.840%

In [ ]:
#predict_submission = model.predict(test)
#sample_submission['price'] = predict_submission
#sample_submission.to_csv('submission.csv', index=False)
#sample_submission.head(3)

# CatBoost + CV

Применим кросс-валидацию с 5 фолдами

In [ ]:
def cat_model(y_train, X_train, X_test, y_test):
    model = CatBoostRegressor(iterations = 10000,
                              learning_rate = 0.25,
                              eval_metric='MAPE',
                              custom_metric=['R2', 'MAE'],
                              random_seed = RANDOM_SEED,)
    model.fit(X_train, y_train,
              #cat_features=cat_features_ids,
              eval_set=(X_test, y_test),
              verbose=False,
              use_best_model=True,
              plot=False)
    
    return(model)

In [ ]:
X = train.values
y = y.values
X_pred = test.values

In [ ]:
N_FOLDS = 5
submissions = pd.DataFrame(0,columns=["sub_1"], index=sample_submission.index) # куда пишем предикты
score_ls = []
splits = list(KFold(n_splits=N_FOLDS, shuffle=True, random_state=RANDOM_SEED).split(X,y))

for idx, (train_idx, test_idx) in tqdm(enumerate(splits), total=N_FOLDS,):
    # use the indexes to extract the folds in the train and validation data
    Xf_train, yf_train, Xf_test, yf_test = X[train_idx], y[train_idx], X[test_idx], y[test_idx]
    # model for this fold
    model = cat_model(yf_train, Xf_train, Xf_test, yf_test)
    # score model on test
    test_predict = model.predict(Xf_test)
    test_score = mape(yf_test, test_predict)
    score_ls.append(test_score)
    print(f"{idx+1} Fold Test MAPE: {mape(yf_test, test_predict)*100:0.3f}%")
    # submissions
    submissions[f'sub_{idx+1}'] = model.predict(X_pred)
    model.save_model(f'catboost_fold_{idx+1}.model')
    
print(f'Mean Score: {np.mean(score_ls)*100:0.3f}%')
print(f'Std Score: {np.std(score_ls)*100:0.3f}%')
print(f'Max Score: {np.max(score_ls)*100:0.3f}%')
print(f'Min Score: {np.min(score_ls)*100:0.3f}%')

Mean MAPE: 2.019%

# Submission

In [ ]:
submissions['blend'] = (submissions.sum(axis=1))/len(submissions.columns)
sample_submission['price'] = submissions['blend'].values
sample_submission.to_csv('submission.csv', index=False)
sample_submission.head(3)

CatBoost с кросс-валидацией дал лучший результат при submission: 12.05451

Проверим теперь, как удаление признаков, сильно коррелирующих с другими, повлияет на MAPE.

In [ ]:
corr_f = ['modelDate', 'date_power5', 'date_power7']
for col in corr_f:
    tmp_train = train.drop([col], axis=1).copy()
    X_train1, X_test1, y_train1, y_test1 = train_test_split(tmp_train, y,
                                                        test_size=0.2, shuffle=True, random_state=42)
    model = cat_model(y_train1, X_train1, X_test1, y_test1)
    test_predict = model.predict(X_test1)
    print(f"{col} Test MAPE: {mape(y_test1, test_predict)*100:0.3f}%")

Без отдельных признаков результат хуже(1.905%, 1.853%, 1.865%), чем базовый(1.840%). Не удаляем их.

# Model 3. Random Forest.

In [ ]:
# A: пар-ры леса по умолчанию
rf = RandomForestRegressor(random_state=42)

rf.fit(X_train, y_train)
test_predict = rf.predict(X_test)
print(f"Test MAPE: {mape(y_test, test_predict)*100:0.3f}%")

! RandomForestRegressor с параметрами по умолчанию показал MAPE = 1.756% -  пока лучший результат  
  Попробуем подобрать значения для некоторых параметров с помощью RandomizedSearchCV

In [ ]:
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [ ]:
# Подобрать пар-ры леса не удалось из-за медленной скорости вычислений 
'''rf = RandomForestRegressor()
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, 
                               n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
rf_random.fit(X_train, y_train)
# Посмотрим, какие гиперпараметры нам предлагают как оптимальные:
rf_random.best_params_'''

In [ ]:
# B: пар-ры леса случайные
rf = RandomForestRegressor(
 n_estimators=300,
 min_samples_split=2,
 min_samples_leaf=1,
 max_features='sqrt',
 max_depth=77,
 bootstrap=True, 
 random_state=42)

rf.fit(X_train, y_train)
test_predict = rf.predict(X_test)
print(f"Test MAPE: {mape(y_test, test_predict)*100:0.3f}%")

MAPE = 1.647 - еще лучше  
RandomForest выигрывает у  CatBoost по MAPE, но проигрывает при submission

# Model 4. Bagging.

In [ ]:
# Посмотрим сначала на точность одного дерева
tree = DecisionTreeRegressor(max_features=int(X_train.shape[1]/3), max_depth=77)
tree.fit(X_train, y_train)
test_predict = tree.predict(X_test)
print(f"DecisionTreeRegressor Test MAPE: {mape(y_test, test_predict)*100:0.3f}%")

In [ ]:
# Теперь обернем деревья в BaggingRegressor
bagging_trees = BaggingRegressor(tree)
bagging_trees.fit(X_train, y_train)
test_predict = bagging_trees.predict(X_test)
print(f"BaggingRegressor(DecisionTrees) Test MAPE: {mape(y_test, test_predict)*100:0.3f}%")

Bagging существенно улучшает качество дерева и дает очень неплохой MAPE = 1.757%

# Model 5. GradientBoosting.

In [ ]:
# Оставляем только числовые признаки и сразу нормируем их для удобства
num_train = train.drop(cat_f, axis=1)
scaler = StandardScaler()
num_train = pd.DataFrame(data=scaler.fit_transform(num_train), columns=num_train.columns)
Xn_train, Xn_test, yn_train, yn_test = train_test_split(num_train, y,
                                                    test_size=0.2, shuffle=True, random_state=42)

In [ ]:
gb = GradientBoostingRegressor(max_depth=7,n_estimators=1500,learning_rate=0.1,random_state=42)
gb.fit(Xn_train, yn_train)
test_predict = gb.predict(Xn_test)
print(f"GradientBoostingRegressor Test MAPE: {mape(yn_test, test_predict)*100:0.3f}%")

Пробуем также на полном наборе признаков.  
GradientBoosting показывает результат MAPE = 1.865% на полном наборе и 2.354% без категориальных.  
На подбор пар-ров не хватает вычислит. ресурсов.

In [ ]:
gb = GradientBoostingRegressor(max_depth=7,n_estimators=1500,learning_rate=0.1,random_state=42)
gb.fit(X_train, y_train)
test_predict = gb.predict(X_test)
print(f"GradientBoostingRegressor Test MAPE: {mape(y_test, test_predict)*100:0.3f}%")

# Model 6. Stacking.

Посмотрим, что даст стекинг с разбиением на фолды.  
Будем использовать немного измененные функции из курса  


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=42)

In [ ]:
cv = KFold(n_splits=5, shuffle=True, random_state=42)

def compute_metric(regr, X_train=X_train, y_train=y_train, X_test=X_test, y_test=y_test):
    regr.fit(X_train, y_train)
    y_test_pred = regr.predict(X_test)
    return np.round(mape(y_test, y_test_pred)*100, 3)

In [ ]:
def compute_meta_feature2(regr, X_train, X_test, y_train, cv, X_pred):    
    
    X_meta_train = np.zeros_like(y_train, dtype=np.float32)    

    splits = cv.split(X_train)
    for train_fold_index, predict_fold_index in splits:
        X_fold_train, X_fold_predict = X_train[train_fold_index], X_train[predict_fold_index]
        y_fold_train = y_train[train_fold_index]
        
        folded_regr = clone(regr)
        folded_regr.fit(X_fold_train, y_fold_train)
        
        X_meta_train[predict_fold_index] = folded_regr.predict(X_fold_predict)
    
    meta_regr = clone(regr)
    meta_regr.fit(X_train, y_train)
    
    X_meta_test = meta_regr.predict(X_test)
    X_meta_pred = meta_regr.predict(X_pred)
    
    return X_meta_train, X_meta_test, X_meta_pred

In [ ]:
def generate_meta_features2(regr_s, X_train, X_test, y_train, cv, X_pred):
   
    features = [compute_meta_feature2(regr, X_train, X_test, y_train, cv, X_pred) for regr in tqdm(regr_s)]    
    stacked_features_train = np.vstack([features_train for features_train, features_test, features_pred in features]).T
    stacked_features_test = np.vstack([features_test for features_train, features_test, features_pred in features]).T
    stacked_features_pred = np.vstack([features_pred for features_train, features_test, features_pred in features]).T
    return stacked_features_train, stacked_features_test, stacked_features_pred

Передаем результаты работы набора из четырех регрессоров на 5ти фолдах каждый в мета-регрессор 

In [ ]:
stacked_features_train, stacked_features_test, stacked_features_pred = generate_meta_features2([
    RandomForestRegressor(n_estimators=300, min_samples_split=2, min_samples_leaf=1, 
                          max_features='sqrt',max_depth=77, bootstrap=True, random_state=42),
    AdaBoostRegressor(random_state=42),
    ExtraTreesRegressor(random_state=42),
    RandomForestRegressor(random_state=42)], X_train, X_test, y_train, cv, X_pred)

regr = RandomForestRegressor(
 n_estimators=300,
 min_samples_split=2,
 min_samples_leaf=1,
 max_features='sqrt',
 max_depth=77,
 bootstrap=True, 
 random_state=42)

print(f'Stacking MAPE = {compute_metric(regr, X_train=stacked_features_train, y_train=y_train, X_test=stacked_features_test, y_test=y_test)}%')

Стекинг дает MAPE = 1.654%.  
При submission результат стекинга очень близок к лучшему(CatBoost+CV): 12.49071

In [ ]:
#predict_submission = regr.predict(stacked_features_pred)#
#sample_submission['price'] = predict_submission
#sample_submission.to_csv('submission.csv', index=False)
#sample_submission.head(3)

# Итоги.  

С авто.ру получено 280 тыс объявлений  
Признаки унифицированы с признаками из теста  
Проведены анализ признаков, оценка их важности и скоррелированности  
Созданы новые признаки и отобран набор для работы с алгоритмами  
Протестировано 5 типов алгоритмов с регрессорами и 1 'наивный' для наглядности  
Лучшую оценку MAPE = 1.647% показал RandomForestRegressor  
Лучший submission выдал CatBoostRegressor с кросс-валидацией